In [0]:
# -*- coding: utf-8 -*-
"""
Pipeline de Processamento de Dados - Amazon Products (LIMITES CORRIGIDOS)
"""

from pyspark.sql import functions as F
import datetime

# ======================================================================
# ETAPA 1: CONFIGURAÇÃO INICIAL
# ======================================================================

def configurar_caminhos(data_arquivo, nome_base_arquivo):
    base_path = "/FileStore/tables/data_ingestion"
    temp_path = "/tmp/ingestao_temporaria"
    output_path = "/FileStore/tables/data_processed"

    nome_arquivo = f"{data_arquivo}_{nome_base_arquivo}.csv"
    return {
        "base_path": base_path,
        "temp_path": temp_path,
        "output_path": output_path,
        "nome_arquivo": nome_arquivo,
        "caminho_origem": f"{base_path}/{nome_arquivo}",
        "caminho_destino": f"{temp_path}/{nome_arquivo}",
        "temp_table_name": f"temp_{nome_base_arquivo}_{data_arquivo}",
        "output_file_name": f"{nome_base_arquivo}_tratado_{data_arquivo}",
        "arquivo_final": f"{output_path}/{nome_base_arquivo}_tratado_{data_arquivo}.csv"
    }

def log_etapa(nome):
    print("\n" + "="*80)
    print(f"📝 {nome}")
    print("="*80)

# Parâmetros principais
data_arquivo = "20250618"
nome_base_arquivo = "amazon"
paths = configurar_caminhos(data_arquivo, nome_base_arquivo)

# ======================================================================
# ETAPA 2: INGESTÃO DE DADOS
# ======================================================================

log_etapa("ETAPA 2: INGESTÃO DE DADOS")

dbutils.fs.mkdirs(paths['temp_path'])

try:
    dbutils.fs.cp(paths['caminho_origem'], paths['caminho_destino'])
    df = (spark.read.csv(paths['caminho_destino'], header=True, sep=",", inferSchema=True))
    df.createOrReplaceTempView(paths['temp_table_name'])
    print(f"✅ Arquivo {paths['nome_arquivo']} ingerido com {df.count():,} registros.")
except Exception as e:
    print(f"❌ ERRO NA INGESTÃO: {e}")
    raise e

# ======================================================================
# ETAPA 3: TRANSFORMAÇÃO E LIMPEZA (LIMITES CORRIGIDOS)
# ======================================================================

log_etapa("ETAPA 3: TRANSFORMAÇÃO E LIMPEZA (LIMITES CORRIGIDOS)")

def transformar_dataframe(df):
    # Normalizar nomes de colunas
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower().replace(" ", "_"))
    
    # Limpeza de colunas numéricas
    colunas_preco = ["discounted_price", "actual_price"]
    for col in colunas_preco:
        df = df.withColumn(col, F.regexp_replace(F.col(col), "[₹,]", "").cast("double"))
    
    df = (df.withColumn("discount_percentage", F.regexp_replace("discount_percentage", "%", "").cast("double"))
            .withColumn("rating", F.col("rating").cast("double"))
            .withColumn("rating_count", F.regexp_replace("rating_count", ",", "").cast("int")))
    
    # CORREÇÃO DOS LIMITES DA CATEGORIZAÇÃO
    df = df.withColumn(
        "price_category",
        # Barato: < 500
        F.when(F.col("discounted_price") < 500, "Barato")
         # Médio: entre 500 e 600 (inclusive)
         .when((F.col("discounted_price") >= 500) & (F.col("discounted_price") <= 600), "Médio")
         # Caro: > 600
         .otherwise("Caro")
    )
    
    # VALIDAÇÃO DETALHADA DAS CATEGORIAS
    print("\n🔥 Verificação das categorias de preço:")
    category_check = df.groupBy("price_category").agg(
        F.min("discounted_price").alias("min_price"),
        F.max("discounted_price").alias("max_price"),
        F.count("*").alias("count")
    ).orderBy("min_price")
    category_check.show()
    
    # Verificação de limites
    print("\n🔍 Limites definidos:")
    print("- Barato: < 500")
    print("- Médio: 500 a 600 (inclusive)")
    print("- Caro: > 600")
    
    # Contagem por categoria para validação
    print("\n🔢 Contagem por categoria:")
    df.groupBy("price_category").count().orderBy("price_category").show()
    
    # Formatação final com preservação de decimais
    df = (df.withColumn("discounted_price", F.format_string("₹%.2f", F.col("discounted_price")))
            .withColumn("actual_price", F.format_string("₹%.2f", F.col("actual_price")))
            .withColumn("discount_percentage", F.format_string("%d%%", F.round(F.col("discount_percentage"), 0).cast("int")))
            .withColumn("rating", F.round("rating", 1))
            .withColumn("rating_count", F.format_number("rating_count", 0)))
    
    return df

try:
    df = spark.sql(f"SELECT * FROM {paths['temp_table_name']}")
    df = transformar_dataframe(df)
    df.persist()
    record_count = df.count()
    print(f"✅ Transformação concluída com {record_count:,} registros.")
    
    print("\n🔍 Colunas após transformação:")
    print(df.columns)
except Exception as e:
    print("❌ ERRO NA TRANSFORMAÇÃO:")
    import traceback
    traceback.print_exc()
    raise e

# ======================================================================
# ETAPA 4: SALVAMENTO DOS DADOS
# ======================================================================

log_etapa("ETAPA 4: SALVAMENTO DOS DADOS")

output_full_path = f"{paths['output_path']}/{paths['output_file_name']}"
dbutils.fs.mkdirs(paths['output_path'])

(df.coalesce(1)
    .write
    .format("csv")
    .option("header", "true")
    .option("sep", "|")
    .option("quote", '"')
    .option("escape", '"')
    .mode("overwrite")
    .save(output_full_path))

output_files = dbutils.fs.ls(output_full_path)
csv_file = next((f for f in output_files if f.name.startswith("part-")), None)

if csv_file:
    dbutils.fs.mv(csv_file.path, paths['arquivo_final'])
    print(f"✅ Arquivo final salvo: {paths['arquivo_final']}")

# ======================================================================
# ETAPA 5: LIMPEZA DE TEMPORÁRIOS
# ======================================================================

log_etapa("ETAPA 5: LIMPEZA DE TEMPORÁRIOS")

spark.sql(f"DROP TABLE IF EXISTS {paths['temp_table_name']}")
dbutils.fs.rm(paths['caminho_destino'], True)
dbutils.fs.rm(output_full_path, True)
print("✅ Recursos temporários limpos.")

# ======================================================================
# ETAPA 6: VALIDAÇÃO FINAL
# ======================================================================

log_etapa("ETAPA 6: VALIDAÇÃO FINAL")

try:
    df_final = (spark.read.csv(paths['arquivo_final'], sep="|", header=True, escape='"'))
    df_final.createOrReplaceTempView(f"{nome_base_arquivo}_tratado")
    total_registros = df_final.count()
    print(f"✅ Validação concluída: {total_registros:,} registros carregados.")
    
    print("\n🔍 Colunas no arquivo final:")
    print(df_final.columns)
    
    # Verificação adicional da categoria
    if "price_category" in df_final.columns:
        print("\n📊 Distribuição final de categorias:")
        # Ordenação correta: Barato -> Médio -> Caro
        ordered_dist = df_final.groupBy("price_category").count().orderBy(
            F.when(F.col("price_category") == "Barato", 1)
             .when(F.col("price_category") == "Médio", 2)
             .otherwise(3)
        )
        ordered_dist.show()
    else:
        print("⚠️ Coluna 'price_category' não encontrada no arquivo final")
except Exception as e:
    print("❌ ERRO NA VALIDAÇÃO FINAL:")
    print(str(e))
    raise e

# ======================================================================
# ETAPA 7: RELATÓRIO FINAL
# ======================================================================

log_etapa("🏁 ETAPA 7: RELATÓRIO FINAL")

print(f"✅ PROCESSO FINALIZADO COM SUCESSO EM {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📂 Origem: {paths['caminho_origem']}")
print(f"📈 Destino: {paths['arquivo_final']}")
print(f"📊 Total de registros: {total_registros:,}")
print("="*80)
print(f"Dica: df = spark.sql('SELECT * FROM {nome_base_arquivo}_tratado')")

# ======================================================================
# ETAPA 8: VISUALIZAÇÃO DA TABELA FINAL
# ======================================================================

log_etapa("👀 ETAPA 8: VISUALIZAÇÃO DA TABELA FINAL")

try:
    print("\n📋 Visualizando os primeiros 20 registros da tabela final:")
    print("="*80)
    
    df_final = spark.read.csv(
        paths['arquivo_final'],
        sep="|",
        header=True,
        escape='"',
        inferSchema=True
    )
    
    df_final.createOrReplaceTempView("visualizacao_final_tmp")
    
    print("🔍 Schema da tabela final:")
    df_final.printSchema()
    
    print("\n🔢 Amostra dos dados (20 registros):")
    display(df_final.limit(20))
    
    print("\n📊 Estatísticas básicas:")
    print(f"- Total de registros: {df_final.count():,}")
    print(f"- Colunas disponíveis: {', '.join(df_final.columns)}")
    
    if "price_category" in df_final.columns:
        print(f"\n📈 Distribuição de categorias de preço:")
        # Ordenação correta: Barato -> Médio -> Caro
        price_dist = df_final.groupBy("price_category").count().orderBy(
            F.when(F.col("price_category") == "Barato", 1)
             .when(F.col("price_category") == "Médio", 2)
             .otherwise(3)
        )
        display(price_dist)
    else:
        print("⚠️ Coluna 'price_category' não encontrada no arquivo final")
        
    print("\n💾 Arquivo salvo em:")
    print(f"dbfs:{paths['arquivo_final']}")
    
    df_final.createOrReplaceTempView(f"{nome_base_arquivo}_tratado_final")
    print(f"\n💡 Use: spark.sql('SELECT * FROM {nome_base_arquivo}_tratado_final')")
    
except Exception as e:
    print(f"❌ Erro na visualização: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("✅ PROCESSO COMPLETO! Use a visualização temporária para análises")
print("="*80)


📝 ETAPA 2: INGESTÃO DE DADOS
✅ Arquivo 20250618_amazon.csv ingerido com 1,465 registros.

📝 ETAPA 3: TRANSFORMAÇÃO E LIMPEZA (LIMITES CORRIGIDOS)

🔥 Verificação das categorias de preço:
+--------------+---------+---------+-----+
|price_category|min_price|max_price|count|
+--------------+---------+---------+-----+
|        Barato|     39.0|    499.0|  571|
|         Médio|    507.0|    600.0|   65|
|          Caro|    609.0|  77990.0|  829|
+--------------+---------+---------+-----+


🔍 Limites definidos:
- Barato: < 500
- Médio: 500 a 600 (inclusive)
- Caro: > 600

🔢 Contagem por categoria:
+--------------+-----+
|price_category|count|
+--------------+-----+
|        Barato|  571|
|          Caro|  829|
|         Médio|   65|
+--------------+-----+

✅ Transformação concluída com 1,465 registros.

🔍 Colunas após transformação:
['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user

product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,price_category
B07JW9H4J1,"Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹399.00,₹1099.00,64%,4.2,"24,269","High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr ,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRANTY: 12 months warranty and friendly customer services, ensures the long-time enjoyment of your purchase. If you meet any question or problem, please don't hesitate to contact us.","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBBSNLYT3ONILA,AHCTC6ULH4XB6YHDY6PCH2R772LQ,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,AENGU523SXMOS7JPDTW52PNNVWGQ,AEQJHCVTNINBS4FKTBGQRQTGTE5Q,AFC3FFC5PKFF5PMA52S3VCHOZ5FQ","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jaspreet singh,Khaja moin,Anand,S.ARUMUGAM","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1KD19VHEDV0OR,R3C02RMYQMK6FC,R39GQRVBUZBWGY,R2K9EDOE15QIRJ,R3OI7YT648TL8I","Satisfied,Charging is really fast,Value for money,Product review,Good quality,Good product,Good Product,As of now seems good","Looks durable Charging is fine tooNo complains,Charging is really fast, good product.,Till now satisfied with the quality.,This is a good product . The charging speed is slower than the original iPhone cable,Good quality, would recommend,https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/81---F1ZgHL._SY88.jpg,Product had worked well till date and was having no issue.Cable is also sturdy enough...Have asked for replacement and company is doing the same...,Value for money",https://m.media-amazon.com/images/W/WEBP_402378-T1/images/I/51UsScvHQNL._SX300_SY300_QL70_FMwebp_.jpg,https://www.amazon.in/Wayona-Braided-WN3LG1-Syncing-Charging/dp/B07JW9H4J1/ref=sr_1_1?qid=1672909124&s=electronics&sr=1-1,Barato
B098NS6PVG,"Ambrane Unbreakable 60W / 3A Fast Charging 1.5m Braided Type C Cable for Smartphones, Tablets, Laptops & other Type C devices, PD Technology, 480Mbps Data Sync, Quick Charge 3.0 (RCT15A, Black)",Computers&Accessories|Accessories&Peripherals|Cables&Accessories|Cables|USBCables,₹199.00,₹349.00,43%,4.0,"43,994","Compatible with all Type C enabled devices, be it an android smartphone (Mi, Samsung, Oppo, Vivo, Realme, OnePlus, etc), tablet, laptop (Macbook, Chromebook, etc)|Supports Quick Charging (2.0/3.0)|Unbreakable – Made of special braided outer with rugged interior bindings, it is ultra-durable cable that won’t be affected by daily rough usage|Ideal Length – It has ideal length of 1.5 meters which is neither too short like your typical 1meter cable or too long like a 2meters cable|Supports maximum 3A fast charging and 480 Mbps data transfer speed|6 months manufacturer warranty from the date of purchase","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBXNGXZJT525AQ,AHONIZU3ICIEHQIGQ6R2VFRSBXOQ,AFPHD2CRPDZMWMBL7WXRSVYWS5JA,AEZ346GX3HJ4O4XNRPHCNHXQURMQ,AEPSWFPNECKO34PUC7I56ITGXR6Q,AHWVEHR5DYLVFTO2KF3IZATFQSWQ,AH4QT33M55677I7ISQOAKEQWACYQ","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Placeholder,BharanI,sonia,Niam","RGIQEG07R9


📊 Estatísticas básicas:
- Total de registros: 1,465
- Colunas disponíveis: product_id, product_name, category, discounted_price, actual_price, discount_percentage, rating, rating_count, about_product, user_id, user_name, review_id, review_title, review_content, img_link, product_link, price_category

📈 Distribuição de categorias de preço:


price_category,count
Barato,571
Médio,65
Caro,829



💾 Arquivo salvo em:
dbfs:/FileStore/tables/data_processed/amazon_tratado_20250618.csv

💡 Use: spark.sql('SELECT * FROM amazon_tratado_final')

✅ PROCESSO COMPLETO! Use a visualização temporária para análises


In [0]:

files = dbutils.fs.ls("/FileStore/tables/data_processed/")
display(files)

path,name,size,modificationTime
dbfs:/FileStore/tables/data_processed/amazon_tratado_20250616.csv,amazon_tratado_20250616.csv,4254350,1750167515000
dbfs:/FileStore/tables/data_processed/amazon_tratado_20250617.csv,amazon_tratado_20250617.csv,4261727,1750170663000
dbfs:/FileStore/tables/data_processed/amazon_tratado_20250618.csv,amazon_tratado_20250618.csv,4261727,1751884434000


In [0]:
#teste
from pyspark.sql import SparkSession

amazon = spark.read.csv("/FileStore/tables/data_processed/amazon_tratado_20250617.csv", header=True, sep=",",inferSchema=True)
amazon.show(5)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_name|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|price_category|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                             B07JW9H4J1|Wayona...|
|                                                                                                                                                       